In [145]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

In [146]:
import pandas as pd
data_merge = pd.read_excel(r'tabladatos.xlsx')
data_merge

,ID_Cliente,ORDERNUMBER,ORDERDATE,Salesperorder,max_year_date,YEAR_ID
0,C001,10136,2003-07-04,102,2003-12-31,2003
1,C001,10178,2003-11-08,413,2003-12-31,2003
2,C001,10397,2005-03-28,172,2005-12-31,2005
3,C002,10280,2004-08-17,551,2004-12-31,2004
4,C002,10293,2004-09-09,292,2004-12-31,2004
...,...,...,...,...,...,...
275,C091,10320,2004-11-03,155,2004-12-31,2004
276,C091,10326,2004-11-09,223,2004-12-31,2004
277,C091,10334,2004-11-19,217,2004-12-31,2004
278,C092,10199,2003-12-01,115,2003-12-31,2003


In [147]:
data_merge['YEAR_ID'] = data_merge['ORDERDATE'].dt.year
data_merge['date_interval'] = data_merge['max_year_date'] - data_merge['ORDERDATE']
data_merge

,ID_Cliente,ORDERNUMBER,ORDERDATE,Salesperorder,max_year_date,YEAR_ID,date_interval
0,C001,10136,2003-07-04,102,2003-12-31,2003,180 days
1,C001,10178,2003-11-08,413,2003-12-31,2003,53 days
2,C001,10397,2005-03-28,172,2005-12-31,2005,278 days
3,C002,10280,2004-08-17,551,2004-12-31,2004,136 days
4,C002,10293,2004-09-09,292,2004-12-31,2004,113 days
...,...,...,...,...,...,...,...
275,C091,10320,2004-11-03,155,2004-12-31,2004,58 days
276,C091,10326,2004-11-09,223,2004-12-31,2004,52 days
277,C091,10334,2004-11-19,217,2004-12-31,2004,42 days
278,C092,10199,2003-12-01,115,2003-12-31,2003,30 days


In [148]:
data_merge['date']=data_merge['date_interval'].astype('timedelta64[D]').astype(int)

In [149]:
data_merge.dtypes

ID_Cliente                object
ORDERNUMBER                int64
ORDERDATE         datetime64[ns]
Salesperorder              int64
max_year_date     datetime64[ns]
YEAR_ID                    int64
date_interval    timedelta64[ns]
date                       int32
dtype: object

In [150]:
rfm = data_merge.groupby(['YEAR_ID', 'ID_Cliente'], as_index = False).agg({'date': 'min', 'ORDERNUMBER': 'count', 'Salesperorder': 'mean'})
rfm.columns = ['YEAR_ID', 'ID_Cliente', 'r', 'f', 'm']
rfm.describe()

,YEAR_ID,r,f,m
count,180.000000,180.000000,180.000000,180.000000
mean,2003.911111,139.700000,1.555556,309.771781
std,0.726889,107.133963,1.182849,154.014693
min,2003.000000,14.000000,1.000000,20.000000
25%,2003.000000,48.750000,1.000000,196.375000
50%,2004.000000,87.500000,1.000000,299.250000
75%,2004.000000,236.250000,2.000000,430.416667
max,2005.000000,360.000000,9.000000,629.000000


In [151]:
rfm.dtypes

YEAR_ID         int64
ID_Cliente     object
r               int32
f               int64
m             float64
dtype: object

In [152]:
def percentil(data):
    five=np.percentile(data,[0,25,50,75,100], interpolation='linear')
    print('min:{0:.6f}'.format(five[0]))
    print('25:{0:.6f}'.format(five[1]))
    print('50:{0:.6f}'.format(five[2]))
    print('75:{0:.6f}'.format(five[3]))
    print('max:{0:.6f}'.format(five[4]))
print('Los 5 percentiles r')
percentil(rfm_gb['r'])
print('Los 5 percentiles f')
percentil(rfm_gb['f'])
print('Los 5 percentiles m')
percentil(rfm_gb['m'])

Los 5 percentiles r
min:14.000000
25:48.750000
50:87.500000
75:236.250000
max:360.000000
Los 5 percentiles f
min:1.000000
25:1.000000
50:1.000000
75:2.000000
max:9.000000
Los 5 percentiles m
min:20.000000
25:196.375000
50:299.250000
75:430.416667
max:629.000000


In [153]:
r_bins = [0,14.000000,48.750000,87.500000,236.250000,360.000000]
f_bins = [0,1.000000, 3.000000,5.000000,7.000000, 9.000000]
m_bins = [0,20.000000, 196.375000, 299.250000, 430.416667,629.000000]
rfm['R'] = pd.cut(rfm['r'], r_bins, labels = [5,4,3, 2, 1])
rfm['F'] = pd.cut(rfm['f'], f_bins, labels = [1, 2, 3,4,5])
rfm['M'] = pd.cut(rfm['m'], m_bins, labels = [1, 2, 3,4,5])
rfm

,YEAR_ID,ID_Cliente,r,f,m,R,F,M
0,2003,C001,53,2,257.5,3,2,3
1,2003,C003,57,2,437.0,3,2,5
2,2003,C005,40,1,447.0,4,1,5
3,2003,C007,55,2,168.0,3,2,2
4,2003,C010,198,1,73.0,2,1,2
...,...,...,...,...,...,...,...,...
175,2005,C083,253,2,463.5,1,2,5
176,2005,C084,253,2,168.0,1,2,2
177,2005,C086,325,1,215.0,1,1,3
178,2005,C087,353,1,469.0,1,1,5


In [154]:
rfm["comb"] = rfm["R"].astype(str) + rfm["F"].astype(str)+ rfm["M"].astype(str)
rfm

,YEAR_ID,ID_Cliente,r,f,m,R,F,M,comb
0,2003,C001,53,2,257.5,3,2,3,323
1,2003,C003,57,2,437.0,3,2,5,325
2,2003,C005,40,1,447.0,4,1,5,415
3,2003,C007,55,2,168.0,3,2,2,322
4,2003,C010,198,1,73.0,2,1,2,212
...,...,...,...,...,...,...,...,...,...
175,2005,C083,253,2,463.5,1,2,5,125
176,2005,C084,253,2,168.0,1,2,2,122
177,2005,C086,325,1,215.0,1,1,3,113
178,2005,C087,353,1,469.0,1,1,5,115


In [159]:
rfm1=rfm.drop(['YEAR_ID', 'R', 'F','M','r','f','m'], axis=1)
rfm1

,ID_Cliente,comb
0,C001,323
1,C003,325
2,C005,415
3,C007,322
4,C010,212
...,...,...
175,C083,125
176,C084,122
177,C086,113
178,C087,115


In [160]:
potenciales=[]
perdidos=[]
derrochadores=[]
leales=[]
nuevos=[]
sin=[]
rfm1['comb']=rfm1['comb'].astype(int)
rfm1['clasificacion']='vacio'
for i in range(len(rfm1)):
    a=rfm1.iloc[i,1]
    if (a==444) or (a==445) or (a==454) or (a==455) or (a==544) or (a==545) or (a==554) or (a==555):
        rfm1.iloc[i,2]='potencial'
    elif (a==111) or (a==112) or (a==113) or (a==121) or (a==122) or (a==123) or (a==131) or (a==132) or (a==133) or (a==211) or (a==212) or (a==213) or (a==221) or (a==222) or (a==223) or (a==231) or (a==232) or (a==233) or (a==311) or (a==312) or (a==313) or (a==321) or (a==322) or (a==323) or (a==331) or (a==332) or (a==333):
        rfm1.iloc[i,2]='perdido'
    elif (a==115) or (a==125) or (a==135) or (a==145) or (a==155) or (a==215) or (a==225) or (a==235) or (a==245) or (a==255) or (a==315) or (a==325) or (a==335) or (a==345) or (a==355) or (a==415) or (a==425) or (a==435) or (a==445) or (a==455) or (a==515) or (a==525) or (a==535) or (a==545) or (a==555):
        rfm1.iloc[i,2]='derrochador'
    elif (a==141) or (a==142) or (a==143) or (a==144) or (a==145) or (a==241) or (a==242) or (a==243) or (a==244) or (a==245) or (a==341) or (a==342) or (a==343) or (a==344) or (a==345) or (a==441) or (a==442) or (a==443) or (a==444) or (a==445) or (a==541) or (a==542) or (a==543) or (a==544) or (a==545) or (a==151) or (a==152) or (a==153) or (a==154) or (a==155) or (a==251) or (a==252) or (a==253) or (a==254) or (a==255) or (a==351) or (a==352) or (a==353) or (a==354) or (a==355) or (a==451) or (a==452) or (a==453) or (a==454) or (a==455) or (a==551) or (a==552) or (a==553) or (a==554):
        rfm1.iloc[i,2]='leal'
    elif (a==411) or (a==412) or (a==413) or (a==414) or (a==421) or (a==422) or (a==423) or (a==424) or (a==431) or (a==432) or (a==433) or (a==434) or (a==511) or (a==512) or (a==513) or (a==514) or (a==521) or (a==522) or (a==523) or (a==524) or (a==531) or (a==532) or (a==533) or (a==534):
        rfm1.iloc[i,2]='nuevos'
    else:
        rfm1.iloc[i,2]='no clasificado'
    

In [161]:
rfm1

,ID_Cliente,comb,clasificacion
0,C001,323,perdido
1,C003,325,derrochador
2,C005,415,derrochador
3,C007,322,perdido
4,C010,212,perdido
...,...,...,...
175,C083,125,derrochador
176,C084,122,perdido
177,C086,113,perdido
178,C087,115,derrochador


In [158]:
rfm1.to_excel('data.xlsx')